Following code is for getting input of query conditions which pass from variabvle queryCondition in single-db-resource-consumption.
Input value are handled by singleDBUtil which is transparent to users.

In [ ]:
# import the required libs
import numpy as np
import pylab as pl
import commonUtil

from scipy.interpolate import spline

#To get all query conditions into a dictionary
queryCondition = commonUtil.handle_query_condition()
ISOK_ALL_PARA = queryCondition['ISOK_ALL_PARA']
REPORT_TYPE = queryCondition['REPORT_TYPE']

DB_SET = queryCondition['DB_SET']
START_TIME = queryCondition['START_TIME']
END_TIME = queryCondition['END_TIME']
INTERVAL = queryCondition['INTERVAL']
GENERAL_ERROR = queryCondition['GENERAL_ERROR']
DB_CONN_ID = DB_SET[0]
#To print the error of query conditon, only print once
for err_id in range(len(queryCondition['GENERAL_ERROR'])):
    print queryCondition['GENERAL_ERROR'][err_id]

Following code is to draw graphs for report Resource Consumption according to the query result, including CPU_SEC, Logical_reads, Physical_reads, and Average Activity Time.

In [ ]:
if(queryCondition['ISOK_ALL_PARA'] and (queryCondition['REPORT_TYPE'] == 'ALL' or queryCondition['REPORT_TYPE'] == 'RESOURCE')):
    #To get data from the target database to assemble a data frame that will be used for the following graphs
    dataframe_hour = %sql select char(date(collected)) as date,\
                          hour(collected) as hours, \
                          sum(total_cpu_usec_delta)/1000000.0 as cpu_sec, \
                          sum(logical_reads_delta) as logical_reads, \
                          sum(PHYSICAL_READS_DELTA) as physical_reads, \
                          sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec, \
                          sum(act_completed_total_delta) as activities \
                          from ibm_dsm_views.throughput_all \
                          where dbconn_id='{DB_CONN_ID}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected),hour(collected) \
                          order by date,hours 
    
    if dataframe_hour.empty:
        print 'For RESOURCE: The query result is empty, please check your query parameters.\n'
    else:
        #To get the hour data as x-axis from dataframe_hour
        x_hour = dataframe_hour['HOURS']
        x_hour_list = list(x_hour)
        #To get date data for forming date + hour format
        date_all = list(dataframe_hour['DATE'].values)
        #To get the CPU_SEC data as y-axis from dataframe_hour
        y_cpu_sec = dataframe_hour['CPU_SEC']
        y_cpu_sec_new = y_cpu_sec
        
        #To get the LOGICAL_READS data as y-axis from dataframe_hour
        y_logical_reads = dataframe_hour['LOGICAL_READS']
        y_logical_reads_new = y_logical_reads
       
        #To get the PHYSICAL_READS data as y-axis from dataframe_hour
        y_physical_reads = dataframe_hour['PHYSICAL_READS']
        y_physical_reads_new = y_physical_reads
        
        #To get the AVG_ACTIVITY_TIME_MSEC data as y-axis from dataframe_hour
        y_avg_act = dataframe_hour['AVG_ACTIVITY_TIME_MSEC']
        y_avg_act_new = y_avg_act
        
        #The tick of x-axis and sorted by asc
        x_ticks = []
        #The label of x-axis
        x_ticks_lables = []
        
        #To get original datetime for after judgement
        ori_datetime_str = commonUtil.get_original_datatime_str(date_all,x_hour_list)
        
        '''
        When the data queried is not equal to the requirement. Filling missing data 
        into the list x_hour_list,date_all and y_ticks.
        '''
        if (len(x_hour_list) < INTERVAL):
            y_cpu_sec_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_cpu_sec, ori_datetime_str)
            y_logical_reads_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_logical_reads, ori_datetime_str)
            y_physical_reads_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_physical_reads, ori_datetime_str)
            #clear date_all
            date_all = []
            #clear x_hour_list
            x_hour_list = []
            y_avg_act_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_avg_act, ori_datetime_str)
            
        commonUtil.format_x_axis(date_all, x_hour_list, x_ticks, x_ticks_lables)
        
        #To declare a Sketchpad as the first graph
        ###First graph###    
        figure_title = 'CPU Usage by Hour\n'
        x_label = 'Hours'
        y_label = 'CPU_SEC(s)'
        fig = commonUtil.declare_figure(x_ticks, x_ticks_lables, figure_title, x_label, y_label)
        #To declare an ax container as the first drawing paper
        ax = fig.add_subplot(111)
        
        commonUtil.draw_point_line(x_ticks, y_cpu_sec_new)

        ###Second graph###
        #To declare a variable to record wherther the value of y_logical_reads greater than 1 million or not
        log_mil = False
        if y_logical_reads.max() > 1000000:
            log_mil = True
            y_logical_reads = y_logical_reads/1000000
        
        phy_mil = False
        if y_physical_reads.max() > 1000000:
            phy_mil = True
            y_physical_reads = y_physical_reads/1000000
        
        figure_title = 'I/O Consumption by Hour\n'

       #To declare a Sketchpad as the second graph
        fig = pl.figure()
        #To declare an ax1 container as the second drawing paper
        ax1 = fig.add_subplot(111)
        #To set the size of the Sketchpad
        if(len(x_ticks) <= 20):
            fig.set_size_inches(12,3)
        elif(len(x_ticks) <= 40):
            fig.set_size_inches(16,3)
        elif(len(x_ticks) <= 60):
            fig.set_size_inches(18,3)
        elif(len(x_ticks) <=100):
            fig.set_size_inches(22,3)

        #To set title for the second graph
        figure_title = 'I/O Consumption by Hour\n'
        pl.title(figure_title, fontsize = 14,fontweight = 'bold')
        #To set x-axis label
        pl.xlabel("Hours", fontsize = 12)
        pl.xticks(x_ticks, x_ticks_lables, rotation=90)
        pl.bar(np.asarray(x_ticks)-0.15, y_logical_reads_new, width=0.3, color = '#2c628b',alpha=0.5)
        #To set y-axis label
        if (log_mil == True):
            pl.ylabel(u'LOGICAL_READS(mil times)')
        else:
            pl.ylabel(u'LOGICAL_READS(times)')
        pl.grid(True, ls = '-.', color = '#a6266e', linewidth = '.5', alpha = 0.3)
        #To set ax1 and ax2 to the same x-axis
        ax2 = ax1.twinx()

        #To draw the second graph using x_hour and y_physical_reads
        pl.bar(np.asarray(x_ticks)+0.15, y_physical_reads_new, width=0.3, color = '#4c78fb', alpha=0.6)
        #To set y-axis lab
        if (phy_mil==True):
            pl.ylabel(u'PHYSICAL_READS(mil times)')
        else:
            pl.ylabel(u'PHYSICAL_READS(times)')

        #To set legend for the ax1 contioner
        ax1.legend(['LOGICAL_READS'], loc='upper right', bbox_to_anchor=(0.5,1.14), ncol=1)
        #To set legend for the ax2 contioner
        ax2.legend(['PHYSICAL_READS'], loc='upper right', bbox_to_anchor=(0.7,1.14), fancybox=True, shadow=True,ncol=1)
        #To set grid format
        pl.grid(True,ls = '-.', color = '#a6266e', linewidth = '0.5', alpha=0.3)
        #To show the second graph
        pl.show()
        
        ###The third graph###
        #To declare a Sketchpad as the third graph
        figure_title = 'Average Activity Time by Hour\n'
        y_label = 'AVG_ACTIVITY_TIME_MSEC'
        
        fig = commonUtil.declare_figure(x_ticks, x_ticks_lables, figure_title, x_label, y_label)
        #To declare an ax container as the first drawing paper
        ax = fig.add_subplot(111)
        commonUtil.draw_point_line(x_ticks, y_avg_act_new)

Following code is to draw graph for report CPU Utilization according to query result. 

In [ ]:
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'CPU')):
    #To get data from the target database to assemble a data frame that will be used for the following graphs
    dataframe_hour = %sql select char(date(collected)) as date, \
                          hour(collected) as hours, \
                          sum(db2_cpu_user_delta) as db2_cpu_user,  \
                          100*sum(db2_cpu_user_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as db2_cpu_user_percent, \
                          sum(db2_cpu_system_delta) as db2_cpu_system,  \
                          100*sum(db2_cpu_system_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as db2_cpu_system_percent, \
                          sum(cpu_user_delta) as cpu_user,  \
                          100*sum(cpu_user_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as cpu_user_percent, \
                          sum(cpu_system_delta) as cpu_system,  \
                          100*sum(cpu_system_delta)/(sum(cpu_user_delta) + sum(cpu_system_delta) + sum(cpu_iowait_delta)+ sum(cpu_idle_delta)) as cpu_system_percent \
                          from IBM_DSM_VIEWS.THROUGHPUT_SYSTEM \
                          where dbconn_id='{DB_CONN_ID}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected),hour(collected) \
                          order by date,hours
    
    if dataframe_hour.empty:
        print 'For CPU: The query result is empty, please check your query parameters.\n'
    else:
        #To get the hour data as x-axis from the data frame
        x_hour = dataframe_hour['HOURS']
        x_hour_list = list(x_hour)
        #To get date data for forming date + hour format
        date_all = list(dataframe_hour['DATE'].values)
        #To get the db2_cpu_user_percent data as top y-axis of the first graph from the data frame
        y_db2_user = dataframe_hour['DB2_CPU_USER_PERCENT']
        y_db2_user_new = y_db2_user
        #To get the db2_cpu_system_percent data as bottom y-axis of the first graph from the data frame
        y_db2_sys = dataframe_hour['DB2_CPU_SYSTEM_PERCENT']
        y_db2_sys_new = y_db2_sys
        #To get the cpu_user_percent data as top y-axis of the second graph from the data frame
        y_cpu_user = dataframe_hour['CPU_USER_PERCENT']
        y_cpu_user_new = y_cpu_user
        #To get the cpu_system_percent data as bottom y-axis of the second graph from the data frame
        y_cpu_sys = dataframe_hour['CPU_SYSTEM_PERCENT']
        y_cpu_sys_new = y_cpu_sys
        
        #The tick of x-axis and sorted by asc
        x_ticks = []
        #The label of x-axis
        x_ticks_lables = []
        
        #To get original datetime used by following judgement
        ori_datetime_str = commonUtil.get_original_datatime_str(date_all,x_hour_list)
        '''
        When the data queried is not equal to the requirement. Filling missing data 
        into the list x_hour_list,date_all and y_ticks.
        '''
        if (len(x_hour_list) < INTERVAL):
            y_db2_user_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_db2_user, ori_datetime_str)
            y_db2_sys_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_db2_sys, ori_datetime_str)
            y_cpu_user_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_cpu_user, ori_datetime_str)
            #clear date_all
            date_all = []
            #clear x_hour_list
            x_hour_list = []
            y_cpu_sys_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_cpu_sys, ori_datetime_str)
        
        commonUtil.format_x_axis(date_all, x_hour_list, x_ticks, x_ticks_lables)
    
        #To declare a Sketchpad including two graphs of left and right distribution
        figure_title = 'CPU Utilization by Hour\n'
        x_label = 'Hours'
        y_label = 'CPU %'
        fig = commonUtil.declare_figure(x_ticks, x_ticks_lables, figure_title, x_label, y_label)
        
        #To declare an ax1 container as the left drawing paper
        ax1 = fig.add_subplot(111)
        
        x_hour = np.asarray(x_ticks)
        #To draw the top y-axis of the upper graph
        pl.bar(x_hour - 0.2, y_db2_user_new, width = 0.4, align = 'center', color = '#60bdae')
        #To draw the bottom y-axis of the upper graph
        pl.bar(x_hour - 0.2, y_db2_sys_new, width=0.4, align = 'center', color = '#a6266e',bottom = y_db2_user_new)
        #To draw the top y-axis of the lower graph
        pl.bar(x_hour + 0.2, y_cpu_user_new, width=0.4, color = '#4c78fb')
        #To draw the bottom y-axis of the lower graph
        pl.bar(x_hour + 0.2, y_cpu_sys_new, width=0.4, color = '#2c628b', bottom = y_cpu_user_new)
        #To set the y-axis labels
        pl.yticks(np.arange(0, 101, 10))

        #To set the legends for the both graphs
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
        ax1.legend(['DB2_CPU_USER_PERCENT', 'DB2_CPU_SYSTEM_PERCENT', 'CPU_USER_PERCENT', 'CPU_SYSTEM_PERCENT'], fontsize = 9, loc = 'upper center', bbox_to_anchor=(0.5,1.06), fancybox = True, shadow = True, ncol = 4)

        #To Show graphs
        pl.show()

Following code is to draw graph for report Memory Usage according to query result.

In [ ]:
#To get data and draw graph by data.
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'MEMORY')):
    #Get data from the target database to assemble a data frame that will be used for the following graph
    dataframe_hour = %sql select char(date(collected)) as date, \
                          hour(collected) as hours, \
                          dec(max(memory_pool_used_gb),17,2) as memory_usage_gb \
                          from IBM_DSM_VIEWS.MEM_DB_TOTAL_USED \
                          where dbconn_id = '{DB_CONN_ID}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected),hour(collected) \
                          order by date,hours
    
    if dataframe_hour.empty:
        print 'For MEMORY: The query result is empty, please check your query parameters.\n'
    else:
        #To get the hour data as x-axis from the data frame and convert the data into a numpy array
        x_hour = dataframe_hour['HOURS'].values
        x_hour_list = list(x_hour)
        date_all = list(dataframe_hour['DATE'].values)
        #To get the memory_usage_gb data as y-axis from the data frame and convert the data into a numpy array
        y_memory = dataframe_hour['MEMORY_USAGE_GB'].values
        y_memory_new = y_memory
        #To define a empty var x_ticks to store x-axis ticks(marks)
        x_ticks = []
        #To define a empty var x_ticks_lables to restore x-axis labels
        x_ticks_lables = []
        
        #To get original datetime used by following judgement
        ori_datetime_str = commonUtil.get_original_datatime_str(date_all, x_hour_list)
        
        '''
        When the data queried is not equal to the requirement. Filling missing data 
        into the list x_hour_list,date_all and y_memory_list.
        '''
        if (len(x_hour_list) < INTERVAL):
            date_all = []
            x_hour_list = []
            y_memory_new = commonUtil.fill_null_data(START_TIME, END_TIME, date_all, x_hour_list, y_memory, ori_datetime_str)
            
        commonUtil.format_x_axis(date_all, x_hour_list, x_ticks, x_ticks_lables)
    
        #To declare a Sketchpad including two graphs of left and right distribution
        figure_title = 'Memory Usage by Hour\n'
        x_label = 'Hours'
        y_label = 'MEMORY_USAGE(GB)'
        fig = commonUtil.declare_figure(x_ticks, x_ticks_lables, figure_title, x_label, y_label)
        ax = fig.add_subplot(111)
        commonUtil.draw_point_line(x_ticks, y_memory_new)